In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
from cobra.util import create_stoichiometric_matrix
from cobra.medium import minimal_medium
import numpy as np
import escher

In [2]:
model = read_sbml_model('model_standard_yeast.xml') # insert model HERE

# The composition of the medium

Let's look at the default media composition

In [3]:
model.medium

{'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_so4_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 999999.0}

**Template:**

In [79]:
medium = model.medium
#model.add_boundary(model.metabolites.pheme_m, type='demand') # adds the exchange reaction for heme
with model:
    solution = model.optimize()
    print("Standard yeast maximum theoretical biomass productivity:", solution.fluxes['BIOMASS_SC5_notrace'], '/h')
    model.medium = medium
    model.objective = model.reactions.DM_pheme_m
    heme_production = model.optimize().objective_value
    print("Standard yeast maximum theoretical productivity of heme ", heme_production, '[mmol gDW^-1 h^-1]')

Standard yeast maximum theoretical biomass productivity: 0.2878657037040176 /h
Standard yeast maximum theoretical productivity of heme  0.5501893939393938 [mmol gDW^-1 h^-1]


# Optimizing growth and heme production at varying glucose concentrations

In [75]:
%%time
medium = model.medium
#model.add_boundary(model.metabolites.pheme_m, type='demand') # adds the exchange reaction for heme
x = []
y = []
i = 0
with model:
    while i < 11: #Optimizes growth for increasing glucose values and stores values in vector x.
        medium['EX_glc__D_e'] = i
        model.medium = medium
        solution = model.optimize()
        #print("Standard yeast maximum theoretical biomass productivity:", solution.fluxes['BIOMASS_SC5_notrace'], '/h')
        x.append(solution.fluxes['BIOMASS_SC5_notrace'])
        i = i + 0.5

    #print(x)
    #print(medium['EX_glc__D_e'])
    i = 0    
    while i < 11: #Optimizes heme prod for increasing glucose values and stores values in vector y.
        medium['EX_glc__D_e'] = i
        model.medium = medium
        model.objective = model.reactions.DM_pheme_m
        heme_production = model.optimize().objective_value
        #print("Standard yeast maximum theoretical productivity of heme ", heme_production, '[mmol gDW^-1 h^-1]')
        y.append(heme_production)
        i = i + 0.1
    #print(y)
    #brug evt: ny_vector = np.array(y) #siden "y" er en liste kan det maaske give problemer naar der skal plottes. Det her laver det np.array

C:\Users\Rasso\anaconda3\envs\carveme_env\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


10.5


C:\Users\Rasso\anaconda3\envs\carveme_env\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Wall time: 3.17 s


The growth- and prod values are stored in lists x and y respectively

In [78]:
for reactions in model.reactions.query('mass','name'):
    print(reactions.id)

BIOMASS_SC5_notrace


In [59]:
model.reactions.BIOMASS_SC5_notrace

Reaction identifier,BIOMASS_SC5_notrace
Name,Biomass SC5 notrace
Memory address,0x02700bd86128
Stoichiometry,1.1348 13BDglcn_c + 0.4588 ala__L_c + 0.046 amp_c + 0.1607 arg__L_c + 0.1017 asn__L_c + 0.2975 asp__L_c + 59.276 atp_c + 0.0447 cmp_c + 0.0066 cys__L_c + 0.0036 damp_c + 0.0024 dcmp_c + 0.0024... 1.1348 1 3 beta D Glucan C6H10O5 + 0.4588 L-Alanine + 0.046 AMP C10H12N5O7P + 0.1607 L-Arginine + 0.1017 L-Asparagine + 0.2975 L-Aspartate + 59.276 ATP C10H12N5O13P3 + 0.0447 CMP C9H12N3O8P +...
GPR,
Lower bound,0.0
Upper bound,999999.0


# Med "slim optimize"
**in case det tager rigtig lang tid at optimere**


In [ ]:
%%time
medium = model.medium
#model.add_boundary(model.metabolites.pheme_m, type='demand') # adds the exchange reaction for heme
x = []
y = []
i = 0
with model:
    while i < 11: #Optimizes growth for increasing glucose values and stores values in vector x.
        medium['EX_glc__D_e'] = i
        model.medium = medium
        solution = model.slim_optimize()
        #print("Standard yeast maximum theoretical biomass productivity:", solution.fluxes['BIOMASS_SC5_notrace'], '/h')
        x.append(model.reactions.BIOMASS_SC5_notrace.flux)
        i = i + 0.05

    #print(x)
    i = 0    
    while i < 11: #Optimizes heme prod for increasing glucose values and stores values in vector y.
        medium['EX_glc__D_e'] = i
        model.medium = medium
        model.objective = model.reactions.DM_pheme_m
        heme_production = model.optimize().objective_value
        #print("Standard yeast maximum theoretical productivity of heme ", heme_production, '[mmol gDW^-1 h^-1]')
        y.append(heme_production)
        i = i + 0.1
    #print(y)
    #brug evt: ny_vector = np.array(y) #siden "y" er en liste kan det maaske give problemer naar der skal plottes. Det her laver det np.array